In [ ]:
# Import Packages

from pyspark.sql import SparkSession
from pyspark.sql.functions import round, add_months
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, BooleanType

spark = SparkSession.builder.appName('Item Price History').master('local[*]').getOrCreate()


In [ ]:
# Import data from 'liquor-sales-data/raw_data/' in ADLS

file_path = "/mnt/liquor-sales-data/raw_data/item_price_history.csv"

schema = StructType([
    StructField("Item Number", StringType(), True),
    StructField("Vendor Number", StringType(), True),
    StructField("Pack", IntegerType(), True),
    StructField("Bottle Volume ml", IntegerType(), True),
    StructField("State Bottle Cost", DoubleType(), True),
    StructField("State Bottle Retail", DoubleType(), True),
    StructField("From Date", DateType(), True),
    StructField("To Date", DateType(), True),
    StructField("Is Current", BooleanType(), False),
    StructField("Price Sequence", IntegerType(), False)
])
df_price_history = spark.read.load(file_path, format='csv', header=True, schema=schema)

In [ ]:
# Transform

# Subtract year column by 543
df_price_history = df_price_history.withColumn('From Date', add_months(df_price_history['From Date'], -543 * 12))
df_price_history = df_price_history.withColumn('To Date', add_months(df_price_history['To Date'], -543 * 12))

# Convert 'Is Current' column to string
df_price_history = df_price_history.withColumn('Is Current', df_price_history['Is Current'].cast('string'))

# Round 'State Bottle Cost' and 'State Botttle Retail' up to 2 decimal figures
df_price_history = df_price_history.withColumn('State Bottle Cost', round(df_price_history['State Bottle Cost'], 2))
df_price_history = df_price_history.withColumn('State Bottle Retail', round(df_price_history['State Bottle Retail'], 2))


df_price_history.show()
df_price_history.printSchema()

+-----------+-------------+----+----------------+-----------------+-------------------+----------+----------+----------+--------------+
|Item Number|Vendor Number|Pack|Bottle Volume ml|State Bottle Cost|State Bottle Retail| From Date|   To Date|Is Current|Price Sequence|
+-----------+-------------+----+----------------+-----------------+-------------------+----------+----------+----------+--------------+
| ITM_941063|      VEN_259|  12|               0|             4.25|               6.38|0928-05-07|0929-11-18|     false|             1|
| ITM_995097|      VEN_322|   6|               0|             8.52|              12.78|0927-12-11|0927-12-11|     false|             1|
| ITM_964590|       VEN_65|  12|               0|              4.8|                7.2|0928-01-29|0928-01-29|      true|             1|
| ITM_934962|      VEN_395|  12|               0|            11.17|              16.76|0928-03-20|0928-09-17|     false|             1|
| ITM_100300|      VEN_255|   6|            1800

In [ ]:
# Export to 'liquor-sales-data/transformed_data/' in ADLS

output_path = "/mnt/liquor-sales-data/transformed_data/item_price_history/"

df_price_history.write.parquet(output_path, mode='overwrite')